# 如何为LLM和聊天模型添加即时工具调用能力
:::注意
部分模型已针对工具调用进行了微调，并提供了专用的工具调用API。通常来说，这类模型在工具调用方面的表现优于未经微调的模型，因此建议在需要工具调用的场景中使用。更多信息请参阅[如何使用聊天模型调用工具](/docs/how_to/tool_calling)指南。
:::
:::info 前提条件
本指南假定您熟悉以下概念：
- [LangChain 工具](/docs/concepts/tools)- [函数/工具调用](https://python.langchain.com/docs/concepts/tool_calling)- [聊天模型](/docs/concepts/chat_models)- [大型语言模型（LLMs）](/docs/concepts/text_llms)
:::
在本指南中，我们将了解如何为聊天模型添加**临时**工具调用支持。如果您使用的模型本身不支持[工具调用](/docs/how_to/tool_calling)，这是一种替代的调用工具的方法。
我们将通过简单地编写一个提示来实现这一点，该提示将引导模型调用适当的工具。以下是逻辑示意图：
![链条](../../static/img/tool_chain.svg)

## 安装设置
我们需要安装以下软件包：

In [ ]:
%pip install --upgrade --quiet langchain langchain-community

如果你想使用LangSmith，请取消以下行的注释：

In [26]:
import getpass
import os
# os.environ["LANGSMITH_TRACING"] = "true"
# os.environ["LANGSMITH_API_KEY"] = getpass.getpass()

您可以选择本指南中提供的任何模型。请注意，这些模型大多已[原生支持工具调用功能](/docs/integrations/chat/)，因此对这些模型采用本文所示的提示策略并无意义，您应当遵循[如何使用聊天模型调用工具](/docs/how_to/tool_calling)指南。
import ChatModelTabs from "@theme/ChatModelTabs";
<ChatModelTabs overrideParams={{openai: {model: "gpt-4"}}} />

为了说明这一概念，我们将通过Ollama使用`phi3`模型——该模型**并不**原生支持工具调用功能。如果您也想使用`Ollama`，请按照[这些说明](/docs/integrations/chat/ollama/)操作。

In [24]:
from langchain_community.llms import Ollama

model = Ollama(model="phi3")

## 创建一个工具
首先，我们创建一个`add`（加法）和`multiply`（乘法）工具。有关创建自定义工具的更多信息，请参阅[本指南](/docs/how_to/custom_tools)。

In [4]:
from langchain_core.tools import tool


@tool
def multiply(x: float, y: float) -> float:
    """Multiply two numbers together."""
    return x * y


@tool
def add(x: int, y: int) -> int:
    "Add two numbers."
    return x + y


tools = [multiply, add]

# Let's inspect the tools
for t in tools:
    print("--")
    print(t.name)
    print(t.description)
    print(t.args)

--
multiply
Multiply two numbers together.
{'x': {'title': 'X', 'type': 'number'}, 'y': {'title': 'Y', 'type': 'number'}}
--
add
Add two numbers.
{'x': {'title': 'X', 'type': 'integer'}, 'y': {'title': 'Y', 'type': 'integer'}}


In [5]:
multiply.invoke({"x": 4, "y": 5})

20.0

## 创建我们的提示
我们需要编写一个提示，明确模型可使用的工具、这些工具的参数以及模型所需的输出格式。在本例中，我们将指示模型输出一个形如 `{"name": "...", "arguments": {...}}` 的 JSON 数据块。

In [6]:
from langchain_core.output_parsers import JsonOutputParser
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.tools import render_text_description

rendered_tools = render_text_description(tools)
print(rendered_tools)

multiply(x: float, y: float) -> float - Multiply two numbers together.
add(x: int, y: int) -> int - Add two numbers.


In [17]:
system_prompt = f"""\
You are an assistant that has access to the following set of tools. 
Here are the names and descriptions for each tool:

{rendered_tools}

Given the user input, return the name and input of the tool to use. 
Return your response as a JSON blob with 'name' and 'arguments' keys.

The `arguments` should be a dictionary, with keys corresponding 
to the argument names and the values corresponding to the requested values.
"""

prompt = ChatPromptTemplate.from_messages(
    [("system", system_prompt), ("user", "{input}")]
)

In [18]:
chain = prompt | model
message = chain.invoke({"input": "what's 3 plus 1132"})

# Let's take a look at the output from the model
# if the model is an LLM (not a chat model), the output will be a string.
if isinstance(message, str):
    print(message)
else:  # Otherwise it's a chat model
    print(message.content)

{
    "name": "add",
    "arguments": {
        "x": 3,
        "y": 1132
    }
}


## 添加输出解析器
我们将使用 `JsonOutputParser` 来将模型输出解析为 JSON 格式。

In [19]:
from langchain_core.output_parsers import JsonOutputParser

chain = prompt | model | JsonOutputParser()
chain.invoke({"input": "what's thirteen times 4"})

{'name': 'multiply', 'arguments': {'x': 13.0, 'y': 4.0}}

:::重要
🎉 太棒了！🎉 我们现在已经指导模型学会如何**请求**调用工具。
现在，让我们创建一些逻辑来实际运行这个工具！好的，请提供需要翻译的英文文本，我会按照标准Markdown格式将其翻译成中文并保持原有结构。以下为示例格式：

**示例输入（英文）**  
```markdown
# Introduction  
This is a **sample** text with [a link](https://example.com).  
- Item 1  
- Item 2  
```

**示例输出（中文）**  
```markdown
# 简介  
这是一段**示例**文本，包含[链接](https://example.com)。  
- 项目1  
- 项目2  
```

请直接提供您的英文内容，我将立即开始翻译。

## 调用工具
既然模型能够请求调用工具，我们需要编写一个能够实际执行调用的函数该工具。
该函数将根据名称选择适当的工具，并将模型选定的参数传递给它。

In [20]:
from typing import Any, Dict, Optional, TypedDict

from langchain_core.runnables import RunnableConfig


class ToolCallRequest(TypedDict):
    """A typed dict that shows the inputs into the invoke_tool function."""

    name: str
    arguments: Dict[str, Any]


def invoke_tool(
    tool_call_request: ToolCallRequest, config: Optional[RunnableConfig] = None
):
    """A function that we can use the perform a tool invocation.

    Args:
        tool_call_request: a dict that contains the keys name and arguments.
            The name must match the name of a tool that exists.
            The arguments are the arguments to that tool.
        config: This is configuration information that LangChain uses that contains
            things like callbacks, metadata, etc.See LCEL documentation about RunnableConfig.

    Returns:
        output from the requested tool
    """
    tool_name_to_tool = {tool.name: tool for tool in tools}
    name = tool_call_request["name"]
    requested_tool = tool_name_to_tool[name]
    return requested_tool.invoke(tool_call_request["arguments"], config=config)

让我们来测试一下 �！

In [21]:
invoke_tool({"name": "multiply", "arguments": {"x": 3, "y": 5}})

15.0

## 让我们一起来整合
让我们将其整合成一个链条，创建一个具备加法和乘法功能的计算器。

In [22]:
chain = prompt | model | JsonOutputParser() | invoke_tool
chain.invoke({"input": "what's thirteen times 4.14137281"})

53.83784653

## 返回工具输入
不仅返回工具输出，同时返回工具输入也很有帮助。我们可以通过 `RunnablePassthrough.assign` 轻松实现这一点，将工具输出分配进去。这会将 RunnablePassthrough 组件的输入（假设是一个字典）原样保留，同时添加一个新的键值，而当前输入中的所有内容仍会完整传递：

In [23]:
from langchain_core.runnables import RunnablePassthrough

chain = (
    prompt | model | JsonOutputParser() | RunnablePassthrough.assign(output=invoke_tool)
)
chain.invoke({"input": "what's thirteen times 4.14137281"})

{'name': 'multiply',
 'arguments': {'x': 13, 'y': 4.14137281},
 'output': 53.83784653}

## 接下来是什么？
本操作指南展示了模型正确输出所有所需工具信息时的“理想路径”。
实际上，如果你使用更复杂的工具，就会开始遇到模型产生的错误，特别是那些未针对工具调用进行微调的模型以及能力较弱的模型。
你需要准备好添加策略来改进模型的输出；例如，
1. 提供少量示例。2. 添加错误处理（例如捕获异常并将其反馈给LLM，要求其修正先前的输出）。